In [ ]:
from flask import Flask, request, jsonify
import pandas as pd
import numpy as np
import joblib
from flask_cors import CORS
from collections import Counter
from statistics import mean

loaded_pipeline = joblib.load('svm_model.joblib')

scaler = loaded_pipeline.named_steps['standardscaler']
svm_classifier = loaded_pipeline.named_steps['svc']

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def filteredPrediction(prediction, confidence_percentages):
    filtered_data = [(pred, confidence) for pred, confidence in zip(prediction, confidence_percentages) if confidence > 30]
    
    # Extract filtered predictions and confidences
    filtered_predictions, filtered_confidences = zip(*filtered_data)
    
    # Count occurrences of each prediction
    prediction_counts = Counter(filtered_predictions)
    
    # Determine the most frequent prediction
    most_frequent_prediction = prediction_counts.most_common(1)[0][0]
    
    # Calculate average confidence
    average_confidence = mean(filtered_confidences) if filtered_confidences else 0
    return most_frequent_prediction,average_confidence

app = Flask(__name__)
CORS(app)

@app.route('/upload', methods=['POST'])
def upload_file():
    if 'file' not in request.files:
        return jsonify({'message': 'No file part'}), 400

    file = request.files['file']
    if file.filename == '':
        return jsonify({'message': 'No selected file'}), 400

    # Do something with the file, e.g., save it to the server
    file.save('uploaded_file.csv')

    # Extract additional form data
    name = request.form.get("name")
    address = request.form.get("address")
    phoneNumber = request.form.get("phoneNumber")
    email = request.form.get("email")
    age = request.form.get("age")
    gender = request.form.get("gender")
    
    # Save additional form data to a separate Excel file
    additional_data = {
        "Name": [name],
        "Address": [address],
        "PhoneNumber": [phoneNumber],
        "Email": [email],
        "Age": [age],
        "Gender": [gender]
    }
    additional_df = pd.DataFrame(additional_data)
    additional_df.to_csv("additional_data.csv", mode='a', index=False, header=not pd.io.common.file_exists("additional_data.csv"))

    return jsonify({'message': 'File uploaded successfully'}), 200

@app.route('/predict', methods=['POST'])
def predict():
    received_data = pd.read_csv('uploaded_file.csv')
    prediction = svm_classifier.predict(received_data).astype(int)
    
    confidence_scores = svm_classifier.decision_function(received_data)
    # print(type(confidence_scores))
    confidence_percentages = np.array([round(float(sigmoid(i) * 100), 2) for i in confidence_scores])
    finalPred,confidence_percentage = filteredPrediction(prediction, confidence_percentages)

    return jsonify({'prediction': finalPred.tolist(), 'confidence': confidence_percentage.tolist()})

if __name__ == '__main__':
    app.run(port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [30/Jan/2025 05:50:42] "OPTIONS /predict HTTP/1.1" 200 -
[2025-01-30 05:50:42,489] ERROR in app: Exception on /predict [POST]
Traceback (most recent call last):
  File "/home/animesh/miniconda3/lib/python3.12/site-packages/flask/app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/animesh/miniconda3/lib/python3.12/site-packages/flask/app.py", line 919, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/animesh/miniconda3/lib/python3.12/site-packages/flask_cors/extension.py", line 194, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
                                                ^^^^^^^^^^^^^^^^^^
  File "/home/animesh/miniconda3/lib/python3.12/site-packages/flask/app.py", line 917, in full_dispatch_request
    rv = self.dispatch_r